Notebook servant à faire l'augmentation du jeu de données de segmentation et faire le détourage des rangées si souhaité.

In [ ]:
import cv2
from matplotlib import pyplot as plt
%matplotlib inline
import albumentations as A

import json
import yaml

from glob import glob
from re import sub
from random import randrange
import os
from PIL import Image
import math
from tqdm import tqdm
import shutil
import numpy as np
import supervision as sv

# Augmentation des données

## Initialisation des chemins utilisés

In [ ]:
inspect = False # Si True affiche les toutes les images augmentées (lent)
do_cut_out = False # Si True créer in dossier avec les images détourées

name = 'S2_6' # Nom pour les dossier finaux (detection_dataset_<name> et cut_out_dataset_<name>)

'''
num_aug: int
Le nombre d'images crée par la première transformation pour chaque image source
Attention, il y a une étape qui fait l'image miroir de toutes les images
Au final on a (nb_img_src * (num_aug + 1)) * 2 images dans le dossier de sortie
'''
num_aug = 6

'''
src_dir: str
Dossier source avec sous dossiers "images" et "labels" et fichier notes.json
'''
src_dir = "source"

'''
seg_res_path: str
Dossier de sortie

seg_res_path
|- train
|   |- images
|   |   |- img1.jpg
|   |   |- img2.jpg
|   |   |- ...
|   |- labels
|       |- img1.txt
|       |- img2.txt
|       |- ...
|- val
|   |- images
|   |   |- img1.jpg
|   |   |- img2.jpg
|   |   |- ...
|   |- labels
|       |- img1.txt
|       |- img2.txt
|       |- ...
|- test
|   |- ...
|- data.yaml
'''
seg_res_path = "segmentation_dataset_" + name

cut_out_res_path = "cut_out_dataset_" + name


## Fonctions de visualisation

In [ ]:
BOX_COLOR = (255, 0, 0) # Red
TEXT_COLOR = (255, 255, 255) # White


def visualize_bbox(img, bbox, color=BOX_COLOR, thickness=2):
    """Ajoute le boite encadrante à l'image

    Args:
        img (np.ndarray): Image source
        bbox (list: int(x_min, y_min, x_max, y_max)): Coordonnées de la boite encadrante
        color (tuple: int, optional): Couleur de la boite. Defaults to BOX_COLOR.
        thickness (int, optional): Epaisseur de la boite. Defaults to 10.

    Returns:
        np.ndarray: Image avec la boite encadrante
    """
    x_min, y_min, x_max, y_max = bbox
    cv2.rectangle(img, (x_min, y_min), (x_max, y_max), color=color, thickness=thickness)
    return img

def visualize_mask(img, mask):
    """Ajoute le mask à l'image

    Args:
        img (np.ndarray): Image source
        mask (np.ndarray): Mask à visualiser

    Returns:
        np.ndarray: Image avec le mask
    """
    color = np.array([0,255,0], dtype='uint8')
    masked_img = np.where(mask[...,None], color, img) # image avec mask plein
    img = cv2.addWeighted(img, 0.8, masked_img, 0.2,0) # image avec mask dilué
    return img

def visualize(image, masks, bboxes, ax = None):
    """Affiche l'image avec ses masks et boites encadrantes

    Args:
        image (np.ndarray): Image source
        masks (list: np.ndarray): Masks à visualiser
        bboxes (list: list): Coordonnées des boites encadrantes
        ax (plt.axes, optional): Position dans la figure matplotlib. Defaults to None.
    """
    img = image.copy()
    for mask in masks:
        img = visualize_mask(img, mask)
    for bbox in bboxes:
        img = visualize_bbox(img, bbox)
    if ax != None:
        ax.imshow(img)
    else:
        plt.figure(figsize=(12, 12))
        #plt.axis('off')
        plt.imshow(img)

## Augmentation

Penser à modifier le chemin d'accès au données source si changements

L'augmentation est faite en 2 étapes, une première avec des augmentation non systématiques (c'est à dire qu'elles ont des probabilité de ne pas ce produire) et une deuxième étape avec des augmentation systématiques (miroir entre autre)

### Mise en place du dossier

In [ ]:
# création des dossier et sous dossiers
splits = []
for split in ['test', 'val', 'train']:
    if split in os.listdir(src_dir):
        splits .append(split)

try:
    os.mkdir(seg_res_path)
except OSError as error:  
    print(error)
for split in splits:
    try:
        os.mkdir(os.path.join(seg_res_path, split))
    except OSError as error:  
        print(error)
    try:
        os.mkdir(os.path.join(seg_res_path, split, 'images'))
    except OSError as error:  
        print(error)
    try:
        os.mkdir(os.path.join(seg_res_path, split, 'labels'))
    except OSError as error:  
        print(error)


### Copie des données val et test

In [ ]:
splits = []
for split in ['test', 'val']:
    if split in os.listdir(src_dir):
        splits .append(split)
        

for split in splits:
    for kind in ['images', 'labels']:
        src_path = os.path.join(src_dir, split, kind)
        targ_path = os.path.join(seg_res_path, split, kind)
        for file in tqdm(os.listdir(src_path), desc = 'file copied'):
            shutil.copy2(os.path.join(src_path, file), targ_path)

### Création du fichier .yaml pour YOLO

In [ ]:
data = {}
try:
    json_f = open(os.path.join(src_dir,'train', "notes.json"))
    json_data = json.load(json_f)

    yaml_f = open(os.path.join(seg_res_path, "data.yaml"), 'w')
    names = {}

    # cat2name sert à lier l'indice de la classe à son nom
    k = 0
    cat2name = [0] * len(json_data['categories'])
    for category in json_data['categories']:
        names[category['id']] = category['name']
        cat2name[k] = category['name']
        k += 1
    
    # ajout des champs de données
    data['names'] = names
    data['nc'] = len(json_data['categories'])
    data['train'] ="./train/images"
    data['val'] = "./val/images"
    data['test'] = "./test/images"

    # écriture dans le fichier yaml
    yaml.dump(data, yaml_f, default_flow_style=False, allow_unicode=True)

    # fermeture des fichiers
    json_f.close()
    yaml_f.close()

except IOError as error:
    print(error)


### Création des données augmentées

In [ ]:
# dossier cibles
img_src_path = os.path.join(src_dir, 'train', 'images')
lab_src_path = os.path.join(src_dir, 'train', 'labels')
img_seg_res_path = os.path.join(seg_res_path, 'train', 'images')
lab_seg_res_path = os.path.join(seg_res_path, 'train', 'labels')
images = glob('*.jpg', dir_fd=img_src_path)

In [ ]:
# transformations
transform1 = A.Compose(
    [
        # Pixels
        A.RandomBrightnessContrast(p=0.2),
        A.RandomGamma(p=0.2),
        A.ISONoise(p=0.2),
        A.GaussNoise(p=0.2),
        A.CLAHE(p=0.2), # add contrast
        A.RandomSunFlare(src_radius = 100, num_flare_circles_upper= 10, p=0.2), # attention au rayon, si trop grand peut entièrement caché une boite
        A.RandomSunFlare(src_radius = 100, num_flare_circles_upper= 10, p=0.2),
        A.RandomSunFlare(src_radius = 100, num_flare_circles_upper= 10, p=0.2), # plusieur pour avoir different angles (ils se forment en ligne)
        
        # Spatial
        A.Rotate(limit=(-10, 10), p=0.3), # voir quel angles sont raisonnables
        A.PixelDropout(dropout_prob=0.01 ,p=0.5),
    ], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['category_ids']), #dans BbowParams on peut ajouter des paramètres de tailles... utile pour les boites qui deviendraient trop petites
)

transform2 = A.Compose(
    [
        A.HorizontalFlip(p=1),
    ], bbox_params=A.BboxParams(format='pascal_voc', label_fields=['category_ids']), #dans BbowParams on peut ajouter des paramètres de tailles... utile pour les boites qui deviendraient trop petites
)

for img_name in tqdm(images, desc = 'images processed'):
    '''
    Lecture des fichiers sources avec récupération des boîtes englobantes
    '''
    image = cv2.imread(os.path.join(img_src_path, img_name)) # lire l'image
    h, w, _ = image.shape
    category_ids = []
    label_name = sub("jpg$", "txt", img_name)
    with open(os.path.join(lab_src_path, label_name), "r") as label_file:
        lines = label_file.readlines()
        polygons = np.empty(len(lines), dtype = object) # Array des polygones des rangées de la figure
        for line_id in range(len(lines)): # Pour chaque annotation (rangée)
            split_line = lines[line_id].split(' ')
            category_ids.append(int(split_line[0]))
            split_line = list(map(float, split_line))
            polygon = np.empty(shape=(len(split_line)//2,2), dtype=np.int32) # Array des sommet du polygon de la rangée
            for i in range(1, len(split_line), 2):
                polygon[i//2] = [split_line[i]*w, split_line[i+1]*h]
            polygons[line_id] = polygon

    masks = [ sv.polygon_to_mask(p,(w,h)) for p in polygons] # Conversion des polygons en masks
    masks = np.array(masks)
    bboxes = [ sv.polygon_to_xyxy(p) for p in polygons] # Conversion des polygons pour avoir la boite encadrante la plus proche

    # copie
    shutil.copy2(os.path.join(img_src_path, img_name), img_seg_res_path)
    shutil.copy2(os.path.join(lab_src_path, label_name), lab_seg_res_path)

    for ind in range(num_aug):
        transformed = transform1(image=image, masks=masks, bboxes=bboxes, category_ids=category_ids)
        transformed_mirror = transform2(image=transformed['image'], masks=transformed['masks'], bboxes=transformed['bboxes'], category_ids=transformed['category_ids'])

        cv2.imwrite(os.path.join(img_seg_res_path, "transformed_" + str(ind) + "_" + img_name), transformed['image'])
        new_label_file = open(os.path.join(lab_seg_res_path, "transformed_" + str(ind) + "_" + label_name), 'w')
        polygons = [ sv.mask_to_polygons(m) for m in transformed['masks'] ]
        for line in range(len(polygons)):
            new_label_file.write(str(transformed['category_ids'][line]))
            for vertice in polygons[line][0]:
                new_label_file.write(" " + str(vertice[0]/w) + " " + str(vertice[1]/h))
            new_label_file.write('\n')
        new_label_file.close()

        cv2.imwrite(os.path.join(img_seg_res_path, "transformed_mirror_" + str(ind) + "_" + img_name), transformed_mirror['image'])
        new_label_file = open(os.path.join(lab_seg_res_path, "transformed_mirror_" + str(ind) + "_" + label_name), 'w')
        polygons = [ sv.mask_to_polygons(m) for m in transformed_mirror['masks'] ]
        for line in range(len(polygons)):
            new_label_file.write(str(transformed_mirror['category_ids'][line]))
            for vertice in polygons[line][0]:
                new_label_file.write(" " + str(vertice[0]/w) + " " + str(vertice[1]/h))
            new_label_file.write('\n')
        new_label_file.close()

    mirror = transform2(image=image, masks=masks, bboxes=bboxes, category_ids=category_ids)
    cv2.imwrite(os.path.join(img_seg_res_path, "mirror_" + img_name), mirror['image'])
    new_label_file = open(os.path.join(lab_seg_res_path, "mirror_" + label_name), 'w')
    polygons = [ sv.mask_to_polygons(m) for m in mirror['masks'] ]
    for line in range(len(polygons)):
        new_label_file.write(str(mirror['category_ids'][line]))
        for vertice in polygons[line][0]:
            new_label_file.write(" " + str(vertice[0]/w) + " " + str(vertice[1]/h))
        new_label_file.write('\n')
    new_label_file.close()


### Check data augmentation results

In [ ]:

if inspect:    
    images = glob('*.jpg', dir_fd=img_seg_res_path)
    num_images = len(images)

    num_grids = math.ceil(num_images / 9)
    for grid in tqdm(range(num_grids), desc = 'grid'):
        fig, axs = plt.subplots(3, 3, figsize=(20, 20))  # Create a 3x3 grid of subplots
        grid_image_paths = images[grid * 9 : (grid + 1) * 9]


        for ax, img_name in zip(axs.flatten(), grid_image_paths):
            image = cv2.imread(img_seg_res_path + '/' + img_name)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            h, w, _ = image.shape
            category_ids = []
            label_name = sub("jpg$", "txt", img_name)
            with open(os.path.join(lab_seg_res_path, label_name), "r") as label_file:
                lines = label_file.readlines()
                polygons = np.empty(len(lines), dtype = object) # Array des polygones des rangées de la figure
                for line_id in range(len(lines)): # Pour chaque annotation (rangée)
                    split_line = lines[line_id].split(' ')
                    category_ids.append(int(split_line[0]))
                    split_line = list(map(float, split_line))
                    polygon = np.empty(shape=(len(split_line)//2,2), dtype=np.int32) # Array des sommet du polygon de la rangée
                    for i in range(1, len(split_line), 2):
                        polygon[i//2] = [split_line[i]*w, split_line[i+1]*h]
                    polygons[line_id] = polygon
            masks = [ sv.polygon_to_mask(p,(w,h)) for p in polygons ] # Conversion des polygons en masks
            masks = np.array(masks)
            bboxes = [ sv.polygon_to_xyxy(p) for p in polygons] # Conversion des polygons pour avoir la boite encadrante la plus proche
            visualize(image, masks, bboxes, ax)
            ax.set_title(img_name)
            ax.axis("off")

        plt.tight_layout()
        plt.show()

In [ ]:

labels_src = glob('*.txt', dir_fd=lab_src_path)
labels_augment = glob('*.txt', dir_fd=lab_seg_res_path)
img_src = glob('*.jpg', dir_fd=img_src_path)
img_augment = glob('*.jpg', dir_fd=img_seg_res_path)

# info source
print("Il y a " + str(len(labels_src)) + " images dans le dossier source:")

label_count = [0] * len(cat2name)

for fname in labels_src:
    with open(lab_src_path + '/' + fname, "r") as label_file:
        for line in label_file:
            split_line = line.split(' ')
            label_count[int(split_line[0])] += 1

for i in range(len(cat2name)):
    print(cat2name[i], " a ", label_count[i], " instances")

# info augmenté
print("------------------------------------------\nIl y a " + str(len(labels_augment)) + " images dans le dossier augmenté:")

label_count = [0] * len(cat2name)

for fname in labels_augment:
    with open(lab_seg_res_path + '/' + fname, "r") as label_file:
        for line in label_file:
            split_line = line.split(' ')
            label_count[int(split_line[0])] += 1

for i in range(len(cat2name)):
    print(cat2name[i], " a ", label_count[i], " instances")

# info dimension moyenne images
print("------------------------------------------\nLes dimension moyennes du jeu source sont:")
mean_w = 0
mean_h = 0

for fname in img_src:
    with Image.open(img_src_path + '/' + fname) as img:
        w, h = img.size
        mean_w += w
        mean_h += h
mean_w /= len(img_src)
mean_h /= len(img_src)

print("mean width = ", mean_w)
print("mean heigh = ", mean_h)

print("------------------------------------------\nLes dimension moyennes du jeu augmenté sont:")
mean_w = 0
mean_h = 0

for fname in img_augment:
    with Image.open(img_seg_res_path + '/' + fname) as img:
        w, h = img.size
        mean_w += w
        mean_h += h
mean_w /= len(img_augment)
mean_h /= len(img_augment)

print("mean width = ", mean_w)
print("mean heigh = ", mean_h)

# Cut Out

In [ ]:
assert(do_cut_out)

In [ ]:
def cut_out(mask, img, bbox):
    """Détoure le mask de l'image et comble la boîte de pixels noirs

    Args:
        mask (np.ndarray): mask composé de 1 et 0 définissant les pixels à détourer
        img (np.ndarray): image de référence
        bbox (list): liste de 4 coordonnées définissant la boite encadrante

    Returns:
        np.array: image de la rangée détourée
    """
    x_min, y_min, x_max, y_max = bbox
    color = np.array([0,0,0], dtype='uint8')
    cut_out_img = np.where(mask[...,None], img, color) # met les pixels correpsondans au 0 du mask en noir
    cut_out_img = cut_out_img[y_min:y_max, x_min:x_max] # coupe l'image au dimensions de la boite encadrante
    return cut_out_img

In [ ]:
# création des dossier et sous dossiers
splits = []
for split in ['test', 'val', 'train']:
    if split in os.listdir(src_dir):
        splits .append(split)

try:
    os.mkdir(cut_out_res_path)
except OSError as error:  
    print(error)
for split in splits:
    try:
        os.mkdir(os.path.join(cut_out_res_path, split))
    except OSError as error:  
        print(error)
    try:
        os.mkdir(os.path.join(cut_out_res_path, split, 'images'))
    except OSError as error:  
        print(error)
    try:
        os.mkdir(os.path.join(cut_out_res_path, split, 'labels'))
    except OSError as error:  
        print(error)

In [ ]:
for split in splits:
    # dossier cibles
    img_src_path = os.path.join(src_dir, split, 'images')
    lab_src_path = os.path.join(src_dir, split, 'labels')
    img_cut_out_res_path = os.path.join(cut_out_res_path, split, 'images')
    lab_cut_out_res_path = os.path.join(cut_out_res_path, split, 'labels')
    images = glob('*.jpg', dir_fd=img_src_path)
    
    for img_name in tqdm(images, desc = 'images processed: ' + split):
        image = cv2.imread(os.path.join(img_src_path, img_name)) # lire l'image
        h, w, _ = image.shape
        category_ids = []
        label_name = sub("jpg$", "txt", img_name)
        with open(os.path.join(lab_src_path, label_name), "r") as label_file:
            lines = label_file.readlines()
            polygons = np.empty(len(lines), dtype = object)
            for line_id in range(len(lines)):
                split_line = lines[line_id].split(' ')
                category_ids.append(int(split_line[0]))
                split_line = list(map(float, split_line))
                polygon = np.empty(shape=(len(split_line)//2,2), dtype=np.int32)
                for i in range(1, len(split_line), 2):
                    polygon[i//2] = [split_line[i]*w, split_line[i+1]*h]
                polygons[line_id] = polygon
        masks = [ sv.polygon_to_mask(p,(w,h)) for p in polygons ]
        masks = np.array(masks)
        bboxes = [ sv.polygon_to_xyxy(p) for p in polygons]

        # découpage et sauvegarde
        for k in range(len(masks)):
            cut_out_img = cut_out(masks[k], image, bboxes[k])
            cv2.imwrite(os.path.join(img_cut_out_res_path, str(k) + '_' + img_name), cut_out_img)